Download Dataset

In [2]:
from pathlib import Path
import os
import pandas as pd
import numpy as np
import cv2
import torch
import sys

In [1]:
DATASET_ROOT_FOLDER = "../datasets/urbanEye"
# ORIGINAL_IMAGES_ROOT = DATSET_ROOT_FOLDER / '/images'
IMAGES_ROOT = f"{DATASET_ROOT_FOLDER}/dataset/images"
DRAWN_ROOT = f"{DATASET_ROOT_FOLDER}/dataset/drawn"
LABEL_ROOT = f"{DATASET_ROOT_FOLDER}/dataset/labels"
TRAIN_CSV_FILE = f"{DATASET_ROOT_FOLDER}/dataset/train.csv"
TEST_CSV_FILE = f"{DATASET_ROOT_FOLDER}/dataset/test.csv"


In [3]:
!pip install gdown

In [ ]:
import gdown


# Google Drive file ID
# file_id = "1ULqYtd9yomeGz53WBhgRdPRFB37ppeDU"
file_id = "16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr"

# Output file name
output = f"{DATASET_ROOT_FOLDER}/dataset22.zip"
if not os.path.isdir(DATASET_ROOT_FOLDER):
    os.makedirs(DATASET_ROOT_FOLDER)

# Download the file
gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr
From (redirected): https://drive.google.com/uc?id=16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr&confirm=t&uuid=63a9daa2-dc8b-45c4-b41b-69c259d7cf8a
To: e:\python\datasets\urbanEye\dataset22.zip
  8%|▊         | 119M/1.44G [00:06<00:54, 24.4MB/s] 

KeyboardInterrupt: 

  9%|▉         | 127M/1.44G [00:20<00:53, 24.4MB/s]

In [ ]:
!gdown "https://drive.google.com/uc?id=16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr"                               


Downloading...
From (original): https://drive.google.com/uc?id=16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr
From (redirected): https://drive.google.com/uc?id=16RDFeGR8NquTh8kCf8IZQa5jxnZnH6Sr&confirm=t&uuid=a0cb6e13-29df-4d5e-a59a-fbf2235e673c
To: e:\python\UrbanEye-scripts-v5\urbaneye-preprocessed-dataset.zip

  0%|          | 0.00/1.44G [00:00<?, ?B/s]
  0%|          | 524k/1.44G [00:00<19:24, 1.24MB/s]
  0%|          | 1.57M/1.44G [00:00<07:20, 3.27MB/s]
  0%|          | 3.67M/1.44G [00:00<03:09, 7.57MB/s]
  1%|          | 8.39M/1.44G [00:00<01:20, 17.7MB/s]
  1%|          | 16.8M/1.44G [00:01<01:17, 18.4MB/s]
  1%|▏         | 20.4M/1.44G [00:01<01:06, 21.4MB/s]
  2%|▏         | 24.1M/1.44G [00:01<01:06, 21.5MB/s]
  2%|▏         | 27.8M/1.44G [00:01<00:57, 24.4MB/s]
  2%|▏         | 30.9M/1.44G [00:01<00:54, 25.8MB/s]
  2%|▏         | 34.1M/1.44G [00:01<00:57, 24.6MB/s]
  3%|▎         | 39.3M/1.44G [00:01<00:46, 30.4MB/s]
  3%|▎         | 44.6M/1.44G [00:02<00:40, 34.7MB/s]
  3%|▎         | 48

In [20]:
import zipfile
import os
# Extract the ZIP file
with zipfile.ZipFile(f"{DATASET_ROOT_FOLDER}/dataset.zip", "r") as zip_ref:
    zip_ref.extractall(DATASET_ROOT_FOLDER)

# Remove the ZIP file to save space
#os.remove("dataset.zip")

print("Dataset extracted successfully!")

Dataset extracted successfully!


In [41]:
#READ CSV files
df_train = pd.read_csv(TRAIN_CSV_FILE, sep=',')
#df_test = df_test.astype({'class':'int', 'xmax':'int', 'xmin':'int', 'ymax':'int', 'ymin':'int'})
x_labeled = df_train.to_numpy()
print(f'labeled features {x_labeled.shape}')
df_test = pd.read_csv(TEST_CSV_FILE, sep=',')
x_unlabeled = df_test.to_numpy()
print(f'test size {x_unlabeled.shape}')

labeled features (19950, 7)
test size (2092, 1)


In [35]:
# CREATE LABEL TEXT FILES
# RESIZE IMAGES
CREATE_FOLDERS = False
SAVE_LABELS = True
SAVE_DRAWN = True
SAVE_RESIZE = True
REMOVE_AFTER_RESIZE = True
TEXT_MAX_COUNTER = -1 #-1 no max
CURRENT_COUNTER = 0

#check and create label folders
if CREATE_FOLDERS:
    folders = ('train', 'eval', 'test')
    foldertypes = (IMAGES_ROOT, LABEL_ROOT, DRAWN_ROOT)
    for fn in folders:
        for folderpath in foldertypes:
            if not os.path.isdir(folderpath):
                os.makedirs(folderpath)
            path = Path(folderpath, fn)
            if not os.path.isdir(path):
                os.makedirs(path)


In [32]:
def plot_one_box(img, x, color=None, label=None, line_thickness=None, Inverted=False):
    
    # Plots one bounding box on image img
    tl = line_thickness or 2 # line thickness
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.circle(img, c1, 10, (255,0,0), 2)
    cv2.circle(img, c2, 10, (255,255,255), 2)
    #print(f'c1 : {c1}, c2 : {c2}')
    cv2.rectangle(img, c1, c2, color, thickness=tl)
    if label:
        tf = tl # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 2, thickness=tf)[0]
        #print(f't_size : {t_size}')
        if Inverted == True:
            c1 = c2
            c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        else:
            c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
            # c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1) # filled
        cv2.putText(img,label,(c1[0], c1[1] - 2),0,tl / 2,[225, 255, 255],thickness=tf,lineType=cv2.LINE_AA,)


In [ ]:
from utils.general import xyxy2xywh


labelled_unique_file_paths = np.unique(x_labeled[:, 1]) #Unique file names
# print(labelled_unique_file_paths)
np.random.shuffle(labelled_unique_file_paths)

# Define split ratios
train_ratio = 0.8
eval_ratio = 0.1
test_ratio = 0.1

# Calculate split sizes
dataset_size = len(labelled_unique_file_paths)
train_counter = int(dataset_size * train_ratio)
eval_counter = int(dataset_size * eval_ratio)
test_counter = dataset_size - (train_counter + eval_counter)  # Ensures full dataset usage

# Create dataset splits
training_paths = ('train', labelled_unique_file_paths[:train_counter])
eval_paths = ('eval', labelled_unique_file_paths[train_counter:train_counter+eval_counter])
test_path = ('test', labelled_unique_file_paths[train_counter+eval_counter:])

print(f'total labelled: {len(labelled_unique_file_paths)}, traing: {train_counter}, eval: {eval_counter}, testing: {test_counter}')
print(training_paths[1].shape)
print(eval_paths[1].shape)
print(test_path[1].shape)

for lbl_group in (training_paths, eval_paths, test_path):
    print(lbl_group[1].size)
    folder = lbl_group[0]
    for file_path in lbl_group[1]:

        #print(file_path)
        path = str(Path(IMAGES_ROOT, file_path))
        if not os.path.exists(path):
             continue
        im = cv2.imread(path)
        #-------RESIZE IMAGE---------#
        im = cv2.resize(im, (int(im.shape[1]/2), int(im.shape[0]/2)), interpolation = cv2.INTER_AREA)
        if SAVE_RESIZE:
            resized_save_to = str(Path(IMAGES_ROOT,folder, file_path))
            cv2.imwrite(resized_save_to, im)
        #----------------------------#
        if SAVE_LABELS:
            label_lines = []
            label_file = str(Path(LABEL_ROOT,folder, os.path.splitext(file_path)[0] + '.txt' ))
        lbl_boxes = x_labeled[np.where(x_labeled[:,1] == file_path)]
        for img_data in lbl_boxes:
            #data in file xmax, xmin, ymax, ymin
            clas = int(img_data[0])
            label = img_data[2]
            xmax = min(img_data[3], im.shape[1])
            xmin = max(img_data[4], 0) #out of bound
            ymax = min(img_data[5], im.shape[0])
            ymin = max(img_data[6], 0)

            if SAVE_LABELS:
                #-------Add Labels---------#
                gn = torch.tensor(im.shape)[[1, 0, 1, 0]]  # normalization gain whwh
                xywh = (xyxy2xywh(torch.tensor([xmin, ymin, xmax, ymax]).view(1, 4)) / gn).view(-1).tolist()  # normalized xywh
                with open(label_file, 'a') as f:
                    f.write(('%g ' * 5 + '\n') % (clas, *xywh)) 
                #yolo_format = np.round(voc_to_yolo_bbox_2([xmin, ymin, xmax, ymax], int(im.shape[1]/2), int(im.shape[0]/2)), 6)
                #label_lines.append(str(clas) + " " + " ".join(str(x) for x in yolo_format))
            if SAVE_DRAWN:
                plot_one_box(im, [xmin, ymin, xmax, ymax], color=(0, 255, 0), label=label, line_thickness=1)

        if SAVE_DRAWN:
            cv2.imwrite(str(Path(DRAWN_ROOT, folder, os.path.basename(path))), im)
        if REMOVE_AFTER_RESIZE:
            os.remove(path)
            
        CURRENT_COUNTER = CURRENT_COUNTER + 1
        if CURRENT_COUNTER % 100 == 0:
            print(f'{folder} - proccessed {CURRENT_COUNTER} images')
            
        if TEXT_MAX_COUNTER > 0 and CURRENT_COUNTER >= TEXT_MAX_COUNTER:
            sys.exit()
    

total labelled: 7874, traing: 6299, eval: 787, testing: 788
(6299,)
(787,)
(788,)
6299
train - proccessed 100 images
train - proccessed 200 images
train - proccessed 300 images
train - proccessed 400 images
train - proccessed 500 images
train - proccessed 600 images
train - proccessed 700 images
train - proccessed 800 images
train - proccessed 900 images
train - proccessed 1000 images
train - proccessed 1100 images
train - proccessed 1200 images
train - proccessed 1300 images
train - proccessed 1400 images
train - proccessed 1500 images
train - proccessed 1600 images
train - proccessed 1700 images
train - proccessed 1800 images
train - proccessed 1900 images
train - proccessed 2000 images
train - proccessed 2100 images
train - proccessed 2200 images
train - proccessed 2300 images
train - proccessed 2400 images
train - proccessed 2500 images
train - proccessed 2600 images
train - proccessed 2700 images
train - proccessed 2800 images
train - proccessed 2900 images
train - proccessed 3000

In [44]:
print(x_unlabeled.size)
unlabelled_unique_file_paths = np.unique(x_unlabeled) #Unique file names
IGNORED_FOLDER_UNLABELLED = f"{DATASET_ROOT_FOLDER}/dataset/ignored"
if not os.path.isdir(IGNORED_FOLDER_UNLABELLED):
    os.makedirs(IGNORED_FOLDER_UNLABELLED)

for file_path in unlabelled_unique_file_paths:
    path = str(Path(IMAGES_ROOT, file_path))
    if not os.path.exists(path):
        continue
    fnew = os.path.join(IGNORED_FOLDER_UNLABELLED, file_path)
    Path(path).replace(fnew)
    
    
    

2092
